# Modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import shapely.geometry as shpg
import matplotlib.pyplot as plt
from matplotlib import cm as colormap
import shapely.ops
from salem import datasets
from cleo import Map
from matplotlib.backends.backend_pdf import PdfPages
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Files

Insert the directories wherever you have unzipped your inventories and you want to have the output

In [ ]:
wgms_dir = '..\\WGMS\\DOI-WGMS-FoG-2014-09'
rgi_dir = '..\\rgi50\\11_rgi50_CentralEurope'
output_dir = '..\\WGMS_LINKS'
f_A = os.path.join(wgms_dir, 'WGMS-FoG-2014-09-A-GENERAL-INFORMATION.csv')
f_EE = os.path.join(wgms_dir, 'WGMS-FoG-2014-09-EE-MASS-BALANCE.csv')
f_rgi = os.path.join(rgi_dir, '11_rgi50_CentralEurope.shp')
f_rgi_links = '\\00_rgi50_links.csv'

# These are the two files already produced 
f_gla = '..\\GLATHIDA_LINKS\\Manual_links_RGI_to_GlaThiDa_ALPS_20160221.csv'
f_lec = '..\\LECLERCQ_LINKS\\Manual_links_Leclercq_to_RGI_ALPS_20160221.csv'

In [2]:
wgms_dir = 'C:\Users\Johannes\Documents\OGGM\\WGMS\\DOI-WGMS-FoG-2014-09'
rgi_dir = 'C:\Users\Johannes\Documents\OGGM\\rgi50\\11_rgi50_CentralEurope'
output_dir = 'C:\Users\Johannes\Documents\OGGM\\test'
f_A = os.path.join(wgms_dir, 'WGMS-FoG-2014-09-A-GENERAL-INFORMATION.csv')
f_EE = os.path.join(wgms_dir, 'WGMS-FoG-2014-09-EE-MASS-BALANCE.csv')
f_rgi = os.path.join(rgi_dir, '11_rgi50_CentralEurope.shp')
f_rgi_links = 'C:\Users\Johannes\Documents\OGGM\\00_rgi50_links.csv'

# These are the two files already produced  
f_gla = 'C:\Users\Johannes\Documents\OGGM\\GLATHIDA_LINKS\\Manual_links_RGI_to_GlaThiDa_ALPS_20160221.csv'
f_lec = 'C:\Users\Johannes\Documents\OGGM\\LECLERCQ_LINKS\\Manual_links_Leclercq_to_RGI_ALPS_20160221.csv'

# Haversine function

In [3]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between one point 
    on the earth and an array of points (specified in decimal degrees)
    """
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371000 # Radius of earth in meters
    return c * r

# Select WGMS IDs in the Alps by lat/lon rectangle

In [4]:
lon_range = [0, 20]
lat_range = [40, 50]

In [5]:
pda = pd.read_csv(f_A, encoding='iso8859_15')
print pda.columns.values

[u'POLITICAL_UNIT' u'NAME' u'WGMS_ID' u'RIVER_BASIN' u'FREE_POSITION'
 u'LOCAL_CODE' u'LOCAL_PSFG' u'GEN_LOCATION' u'SPEC_LOCATION' u'LATITUDE'
 u'LONGITUDE' u'PRIM_CLASSIFIC' u'FORM' u'FRONTAL_CHARS' u'EXPOS_ACC_AREA'
 u'EXPOS_ABL_AREA' u'PARENT_GLACIER' u'REMARKS' u'GEO-REGION_CODE'
 u'GEO-SUBREGION_CODE']


In [6]:
alps_ids = pda[(pda.LONGITUDE >= lon_range[0]) & (pda.LONGITUDE <= lon_range[1]) &
               (pda.LATITUDE >= lat_range[0]) & (pda.LATITUDE <= lat_range[1])].WGMS_ID.values

# Select glaciers with more than 5 mass balance measurements in WGMS FoG

In [7]:
pdee = pd.read_csv(f_EE, encoding='iso8859_15')
pdee = pdee[pdee.WGMS_ID.isin(alps_ids)]

In [8]:
# Where we have yearly measures and no altitude range
pdee = pdee[pdee.LOWER_BOUND.isin([9999])]

In [9]:
gp_id = pdee.groupby('WGMS_ID')
ids_5 = []
for wgmsid, group in gp_id:
    if np.sum(np.isfinite(group.ANNUAL_BALANCE.values)) >= 5:
        ids_5.append(wgmsid)
len(ids_5)

42

In [10]:
alps_ids = ids_5

# Read the RGI file

In [11]:
pdrgi = gpd.read_file(f_rgi)
pdrgi.head(1)

,Area,Aspect,BgnDate,CenLat,CenLon,EndDate,GLIMSId,GlacType,Lmax,Name,O1Region,O2Region,RGIFlag,RGIId,Slope,Zmax,Zmed,Zmin,geometry
0,0.122,334,20030799,47.4949,13.5987,20030999,G013599E47495N,0099,461,None,11,1,0909,RGI50-11.00001,19.9,2338,2277,2191,"POLYGON ((13.600350154 47.49330088799999, 13.5..."


# Read also the GlaThiDa link file

In [12]:
gla_links = pd.read_csv(f_gla)
gla_links.head(1)

IOError: File C:\Users\Johannes\Documents\OGGM\GLATHIDA_LINKS\Manual_links_RGI_to_GlaThiDa_ALPS_20160221.csv does not exist

# Read also the Leclercq link file

In [ ]:
lec_links = pd.read_csv(f_lec)
lec_links.head(1)

# Create an overview plot with ALL glaciers

In [ ]:
curr = 1  # current plot
total = len(alps_ids)  # total number of plots ()

dummyline = plt.Line2D((0,1),(0,0), color='w', linewidth=0, linestyle='')  # for dummy 'category' entry

with PdfPages(os.path.join(output_dir, 'WGMS_Glaciers_Alps_ALL.pdf')) as pdf:
    for gid in alps_ids:
        glacier = pda[pda.WGMS_ID == gid].iloc[0]
        lon, lat = glacier.LONGITUDE, glacier.LATITUDE
        pdrgi['DIST'] = haversine(lon, lat, pdrgi.CenLon.values, pdrgi.CenLat.values)
        sortrgi = pdrgi.sort(columns='DIST')
        
        # For GoogleMap we need a lon lat range to generate the map
        mmlon = [lon, lon]
        mmlat = [lat, lat]
        
        for i in np.arange(0,5):
            rgig = sortrgi.iloc[i]
            # In case the glacier is a MultiPolygon we account for this here:
            if rgig.geometry.type == 'Polygon':
                x, y = rgig.geometry.exterior.xy
            elif rgig.geometry.type == 'MultiPolygon':
                # buffer is necessary as some multi-polygons are self-intersecting
                allparts = [p.buffer(0) for p in rgig.geometry] 
                rgig.geometry = shapely.ops.cascaded_union(allparts)
                x, y = rgig.geometry.exterior.xy

            mmlon = [np.min(np.append(mmlon, x)), np.max(np.append(mmlon, x))]
            mmlat = [np.min(np.append(mmlat, y)), np.max(np.append(mmlat, y))]
        
        # Make a local map where to plot the polygons
        local = datasets.GoogleVisibleMap(x=mmlon, y=mmlat) # also possible:  maptype='terrain'
        local_map = Map(local.grid, countries=False, nx=640)
        local_map.set_lonlat_countours()
        
        
        # Prepare the figure
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        ax1.set_title('{}: '.format(gid) + glacier.POLITICAL_UNIT + '-' + glacier.NAME)
        ax2.set_title("Smallest Haversine distance: %.2f m" % (haversine(lon, lat, sortrgi.iloc[0].CenLon, sortrgi.iloc[0].CenLat)))

        # Plot glaciers
        colors = ['red', 'orange', 'green', 'blue', 'purple', 'magenta']
        for i in np.arange(0,5):
            rgig = sortrgi.iloc[i]
            # In case the glacier is a MultiPolygon we (again) account for this here:
            if rgig.geometry.type == 'Polygon':
                x, y = rgig.geometry.exterior.xy
            elif rgig.geometry.type == 'MultiPolygon':
                # buffer is necessary as some multi-polygons are self-intersecting
                allparts = [p.buffer(0) for p in rgig.geometry] 
                rgig.geometry = shapely.ops.cascaded_union(allparts)
                x, y = rgig.geometry.exterior.xy
            
            #  print centroid of matching glacier
            if i == 0:
                local_map.set_geometry(shpg.Point(rgig.CenLon, rgig.CenLat), edgecolor='k', marker='x', linewidth=4, markersize=100, zorder=50, text='RGI')
            
            # RGI polygon label
            if rgig.Name == None:
                plabel =  str(rgig.RGIId)+'\n'+str(rgig.Area)+'km2'
            else:
                plabel =  str(rgig.RGIId)+'\n'+str(rgig.Area)+'km2\n'+str(rgig.Name)
            
            local_map.set_geometry(rgig.geometry.exterior, color=colors[i], linewidth=3, label=plabel) #adjusted for RGI 5.0
        local_map.set_geometry(shpg.Point(rgig.CenLon, rgig.CenLat), c='k', marker='x', markersize=30, zorder=51) #again adjusted fpr RGI 5.0
        # Plot the GlaThiDa point
        local_map.set_geometry(shpg.Point(lon, lat), color='g', marker='x', linewidth=4, markersize=100, zorder=50, text='WGMS')
        
        local_map.set_rgb(local.get_vardata())
        local_map.visualize(ax=ax1, addcbar=False)

        local = datasets.GoogleVisibleMap(x=mmlon, y=mmlat, maptype='terrain')
        local_map.set_rgb(local.get_vardata())
        local_map.visualize(ax=ax2, addcbar=False)
        plt.subplots_adjust(left=0.04, right=0.80, top=0.94, bottom=0.07)
        plt.legend(bbox_to_anchor=(1.02, 1.), fontsize=18, loc=2, borderaxespad=0, frameon=False)

        pdf.savefig(fig)
        plt.close()
        
        if curr % 5 == 0:
            print "%s / %s plots done." % (curr, total)
        curr += 1

# Try and automatically link WGMS IDs to all known inventories

In [ ]:
manual_links = pda.copy()
manual_links = manual_links[manual_links.WGMS_ID.isin(alps_ids)]
manual_links = manual_links[['POLITICAL_UNIT', 'NAME', 'WGMS_ID', 'LATITUDE', 'LONGITUDE']]
manual_links['GlaThiDa_ID'] = np.nan
manual_links['RGI_ID'] = np.nan
manual_links['Leclercq_ID'] = np.nan
manual_links['remark'] = np.nan

In [ ]:
for gid in manual_links.WGMS_ID.values:
    
    # Link Leclercq, if possible
    if gid in lec_links.WGMS_ID.values:
        manual_links.Leclercq_ID[manual_links.WGMS_ID == gid] = lec_links.ID[lec_links.WGMS_ID == gid].iloc[0]
        
    # Use Graham to link with RGI, if possible
    if gid in gra_links.FoGId.values:
        manual_links.RGI_ID[manual_links.WGMS_ID == gid] = gra_links.RGIId[gra_links.FoGId == gid].iloc[0]
    
    # Check if some WGMS glaciers that are not in Graham's file and still need an RGI ID can be filled with Leclercq's links (which also contain RGI IDs)
    if (not pd.isnull(manual_links.Leclercq_ID[manual_links.WGMS_ID == gid].iloc[0]) and pd.isnull(manual_links.RGI_ID[manual_links.WGMS_ID == gid].iloc[0])):
        manual_links.RGI_ID[manual_links.WGMS_ID == gid] = lec_links.RGI_ID[lec_links.ID == manual_links.Leclercq_ID[manual_links.WGMS_ID == gid].iloc[0]].iloc[0]

        
# Link WGMS to GlaThiDa via RGI (GlaThiDa and RGi have already been linked manually)
for rgiid in manual_links.RGI_ID.values:
    if not pd.isnull(rgiid) and (rgiid in gla_links.RGI_equivalent.values):
        manual_links.GlaThiDa_ID[manual_links.RGI_ID == rgiid] = gla_links.GlaThiDa_ID[gla_links.RGI_equivalent == rgiid].iloc[0]
        

manual_links

In [ ]:
manual_links.to_csv(os.path.join(output_dir, 'Automated_links_WGMS_to_RGI_GlaThiDa_Leclercq_ALPS.csv'))